In [1]:
# Install torch geometric
import os
if 'IS_GRADESCOPE_ENV' not in os.environ:
  !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.0.1%2Bcu118.html
  !pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-2.0.1%2Bcu118.html
  !pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1%2Bcu118.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1%2Bcu118.html


In [2]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch_geometric
torch_geometric.__version__

'2.3.1'

In [4]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import torch
from collections import defaultdict
from typing import Any, Dict, Iterable, List, Optional, Tuple, Union

import scipy.sparse
from torch import Tensor
from torch.utils.dlpack import from_dlpack, to_dlpack

from torch_geometric.utils.num_nodes import maybe_num_nodes

In [5]:
train=pd.read_csv('/Users/akshaj.g/Desktop/ml/social network /FacebookRecruiting/train.csv')

In [6]:
edge_index = torch.tensor(train.values).T
num_nodes = edge_index.max().item()
print(num_nodes)

1862220


In [7]:
edge_index.shape

torch.Size([2, 9437519])

In [8]:
def to_scipy_sparse_matrix(
    edge_index: Tensor,
    edge_attr: Optional[Tensor] = None,
    num_nodes: Optional[int] = None,
) -> scipy.sparse.coo_matrix:
    row, col = edge_index.cpu()

    if edge_attr is None:
        edge_attr = torch.ones(row.size(0))
    else:
        edge_attr = edge_attr.view(-1).cpu()
        assert edge_attr.size(0) == row.size(0)

    N = maybe_num_nodes(edge_index, num_nodes)
    out = scipy.sparse.coo_matrix(
        (edge_attr.numpy(), (row.numpy(), col.numpy())), (N, N))
    return out


In [9]:
adj_mat=to_scipy_sparse_matrix(edge_index)

In [10]:
!pip3 install tensordict
import torch.nn as nn
from tensordict import TensorDict
from tensordict.nn import TensorDictModule, TensorDictSequential

In [11]:
node_features=defaultdict(torch.Tensor)

In [12]:
def node_degree(edge_list, adj_mat, num_nodes):
  degree=torch.tensor(adj_mat.sum(axis=1).A1, dtype=torch.long)
  degree = torch.where(degree > 0, degree, torch.zeros_like(degree))
  if degree.shape[0] < num_nodes:
    degree = torch.cat((degree, torch.zeros(num_nodes - degree.shape[0], dtype=torch.long)))

  return degree


In [13]:
degree_t=node_degree(edge_index,adj_mat,num_nodes)

In [14]:
import torch
import torch_scatter
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType,
                                    OptTensor)

from torch.nn import Parameter, Linear
from torch_sparse import SparseTensor, set_diag
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

class GNNStack(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, args, emb=False):
        super(GNNStack, self).__init__()
        conv_model = self.build_conv_model(args.model_type)
        self.convs = nn.ModuleList()
        self.convs.append(conv_model(input_dim, hidden_dim))
        assert (args.num_layers >= 1), 'Number of layers is not >=1'
        for l in range(args.num_layers-1):
            self.convs.append(conv_model(args.heads * hidden_dim, hidden_dim))


        self.post_mp = nn.Sequential(
            nn.Linear(args.heads * hidden_dim, hidden_dim), nn.Dropout(args.dropout),
            nn.Linear(hidden_dim, output_dim))

        self.dropout = args.dropout
        self.num_layers = args.num_layers

        self.emb = emb

    def build_conv_model(self, model_type):
        if model_type == 'GraphSage':
            return GraphSage
        elif model_type == 'GAT':
            return GAT
        
    def forward(self, x, pos_edge_index, neg_edge_index):
        for i in range(self.num_layers):
            x = self.convs[i](x, pos_edge_index, neg_edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.post_mp(x)

        if self.emb:
            return x

        return F.log_softmax(x, dim=1)
    
    def get_node_embeddings(self, x):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=False)  # Set training to False to get deterministic output
        return x 
    
    def loss(self, pred, label):
        return F.nll_loss(pred, label)

In [15]:
class GraphSage(MessagePassing):
    
    def __init__(self, in_channels, out_channels, normalize = True,
                 bias = False, **kwargs):  
        super(GraphSage, self).__init__(**kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.normalize = normalize

        self.lin_l = None
        self.lin_r = None
        self.lin_l = nn.Linear(self.in_channels, self.out_channels)
        self.lin_r = nn.Linear(self.in_channels, self.out_channels)
        self.reset_parameters()

    def reset_parameters(self):
        self.lin_l.reset_parameters()
        self.lin_r.reset_parameters()

    '''def forward(self, x, pos_edge_index, neg_edge_index, size=None):
        # Propagate messages for positive edges
        x_j_pos = self.propagate(pos_edge_index, x=x)
        
        # Propagate messages for negative edges
        x_j_neg = self.propagate(neg_edge_index, x=x)
        
        # Create a mask tensor for missing nodes in pos_edge_index
        mask_pos = torch.zeros(x.size(0), device=x.device)
        mask_pos[pos_edge_index[0]] = 1.0
        
        # Create a mask tensor for missing nodes in neg_edge_index
        mask_neg = torch.zeros(x.size(0), device=x.device)
        mask_neg[neg_edge_index[0]] = 1.0
        
        # Apply the masks to embeddings
        x_j_pos = x_j_pos * mask_pos.unsqueeze(1)
        x_j_neg = x_j_neg * mask_neg.unsqueeze(1)
        
        # Perform the linear transformations
        out = self.lin_l(x) + self.lin_r(x_j_pos) - self.lin_r(x_j_neg)
        
        if self.normalize:
            out = F.normalize(out, p=2)
        return out'''
    
    def forward(self,x,pos_edge_index,neg_edge_index,size = None):
        out = None
        x_j_pos = self.propagate(pos_edge_index,x=x)
        x_j_neg = self.propagate(neg_edge_index,x=x)
        out = self.lin_l(x) + self.lin_r(x_j_pos) - self.lin_r(x_j_neg)
        if self.normalize:
            out = F.normalize(out, p=2)
        return out

    def message(self, x_j):

        out = None
        out = x_j
        return out

    def aggregate(self, inputs, index, dim_size=None):
        out = None
        node_dim = self.node_dim
        out = torch_scatter.scatter(inputs, index, dim=node_dim, reduce='mean')
        return out


In [16]:
class GAT(MessagePassing):

    def __init__(self, in_channels, out_channels, heads = 2,
                 negative_slope = 0.2, dropout = 0., **kwargs):
        super(GAT, self).__init__(node_dim=0, **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.negative_slope = negative_slope
        self.dropout = dropout

        self.lin_l = None
        self.lin_r = None
        self.att_l = None
        self.att_r = None

        self.lin_l = nn.Linear(self.in_channels, self.heads*self.out_channels)
        self.lin_r = self.lin_l
        self.att_l = nn.Parameter(torch.zeros(self.heads, self.out_channels))
        self.att_r=self.att_l
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.lin_l.weight)
        nn.init.xavier_uniform_(self.lin_r.weight)
        nn.init.xavier_uniform_(self.att_l)
        nn.init.xavier_uniform_(self.att_r)

    def forward(self, x, edge_index, size = None):

        H, C = self.heads, self.out_channels
        x_l=self.lin_l(x).view(-1, H, C)
        x_r=self.lin_r(x).view(-1, H, C)
        alpha_l=self.att_l*x_l
        alpha_r=self.att_r*x_r
        out=self.propagate(edge_index, x=(x_l,x_r), alpha=(alpha_l, alpha_r), size=size).view(-1,H*C)
        return out


    def message(self, x_j, alpha_j, alpha_i, index, ptr, size_i):
        alpha=(alpha_j + alpha_i).squeeze(-1)
        alpha=F.leaky_relu(alpha, negative_slope=0.2)
        if ptr:
            alpha = F.softmax(alpha,ptr)
        else:
            alpha=torch_geometric.utils.softmax(alpha, index)
        alpha=F.dropout(alpha, p=self.dropout, training=self.training)
        out=x_j * alpha
        return out


    def aggregate(self, inputs, index, dim_size = None):
        node_dim = self.node_dim
        out = torch_scatter.scatter(inputs, index, node_dim, dim_size=dim_size, reduce='sum')
        return out

In [17]:
positive_edges=edge_index

In [18]:
test=pd.read_csv('/Users/akshaj.g/Desktop/ml/social network /FacebookRecruiting/test.csv')

In [19]:
test_index = torch.tensor(test.values).T
test_index.shape
edge_index.shape

torch.Size([2, 9437519])

In [20]:
from torch_geometric.utils import negative_sampling

In [21]:
neg_edges=negative_sampling(edge_index)

In [22]:
neg_edges.shape

torch.Size([2, 9437519])

In [23]:
edge_label = torch.tensor(([1.] * 100 + [0.] * 100),requires_grad=True)
edge_label.shape

torch.Size([200])

In [24]:
max_node_index = max(positive_edges.max(), neg_edges.max())
num_nodes = max_node_index + 1
x = torch.rand(num_nodes, 16)

In [25]:
loss_fn = nn.BCELoss()
sigmoid = nn.Sigmoid()

def accuracy(pred, label):
  accu = 0.0
  y_hat = (pred>0.5).type(torch.LongTensor)
  accu = torch.mean((label==y_hat).type(torch.FloatTensor))
  return accu

In [26]:
import torch.optim as optim

def build_optimizer(args, params):
    weight_decay = args.weight_decay
    filter_fn = filter(lambda p : p.requires_grad, params)
    if args.opt == 'adam':
        optimizer = optim.Adam(filter_fn, lr=args.lr, weight_decay=weight_decay)
    elif args.opt == 'sgd':
        optimizer = optim.SGD(filter_fn, lr=args.lr, momentum=0.95, weight_decay=weight_decay)
    elif args.opt == 'rmsprop':
        optimizer = optim.RMSprop(filter_fn, lr=args.lr, weight_decay=weight_decay)
    elif args.opt == 'adagrad':
        optimizer = optim.Adagrad(filter_fn, lr=args.lr, weight_decay=weight_decay)
    if args.opt_scheduler == 'none':
        return None, optimizer
    elif args.opt_scheduler == 'step':
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.opt_decay_step, gamma=args.opt_decay_rate)
    elif args.opt_scheduler == 'cos':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.opt_restart)
    return scheduler, optimizer

In [27]:
'''import time
import networkx as nx
import numpy as np
import torch
import torch.optim as optim
from tqdm import trange
import pandas as pd
import copy

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.nn as pyg_nn

import matplotlib.pyplot as plt


def train(positive_edges,neg_edges,args,batch_size):
#building the model 

    model = GNNStack(16, args.hidden_dim, 8,args)
    
    scheduler, opt = build_optimizer(args, model.parameters())

#starting the training 

    for epoch in trange(args.epochs, desc="Training", unit="Epochs"):
        total_loss = 0
        model.train()
        
#shuffle the edges
        p_idx = torch.randperm(positive_edges.size(1))
        p=positive_edges[:,p_idx]
        p=torch.tensor(p)
        n_idx = torch.randperm(neg_edges.size(1))
        n=neg_edges[:,n_idx]
        n=torch.tensor(n)
#batch the edges
        p_b = p[:,:batch_size]
        n_b = n[:,:batch_size]
        batch_edge = torch.cat([p_b,n_b], dim=1)
        print(batch_edge.shape)
#making pred and labels 
        opt.zero_grad()
        pred = model(batch_edge)
        edge_label = torch.cat([torch.ones(batch_size), torch.zeros(batch_size)], dim=0)
        train_edge_mask = batch_edge.train_mask
        train_pred = pred[train_edge_mask]
        train_labels = edge_label[train_edge_mask]
#loss is binary cross loss 
            
        loss = F.binary_cross_entropy_with_logits(train_pred, train_labels.view(-1, 1))
        loss.backward()
        opt.step()

        acc = accuracy(edge_feature , edge_label)
        print(emb)
        print(f"Epoch: {epoch+1}, Loss: {loss.item():.4f}, Accuracy: {acc}")
    node_embeddings = emb.detach()
    return node_embeddings'''

'import time\nimport networkx as nx\nimport numpy as np\nimport torch\nimport torch.optim as optim\nfrom tqdm import trange\nimport pandas as pd\nimport copy\n\nfrom torch_geometric.datasets import TUDataset\nfrom torch_geometric.datasets import Planetoid\nfrom torch_geometric.data import DataLoader\n\nimport torch_geometric.nn as pyg_nn\n\nimport matplotlib.pyplot as plt\n\n\ndef train(positive_edges,neg_edges,args,batch_size):\n#building the model \n\n    model = GNNStack(16, args.hidden_dim, 8,args)\n    \n    scheduler, opt = build_optimizer(args, model.parameters())\n\n#starting the training \n\n    for epoch in trange(args.epochs, desc="Training", unit="Epochs"):\n        total_loss = 0\n        model.train()\n        \n#shuffle the edges\n        p_idx = torch.randperm(positive_edges.size(1))\n        p=positive_edges[:,p_idx]\n        p=torch.tensor(p)\n        n_idx = torch.randperm(neg_edges.size(1))\n        n=neg_edges[:,n_idx]\n        n=torch.tensor(n)\n#batch the edges\n

In [28]:

import time
import networkx as nx
import numpy as np
import torch
import torch.optim as optim
from tqdm import trange
import pandas as pd
import copy

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.nn as pyg_nn

import matplotlib.pyplot as plt

def train(x, positive_edge, neg_edges, args, batch_size):
    # Build the model
    model = GNNStack(16, args.hidden_dim, 8, args)
    
    scheduler, opt = build_optimizer(args, model.parameters())

    # Starting the training
    for epoch in trange(args.epochs, desc="Training", unit="Epochs"):
        total_loss = 0
        model.train()

        # Shuffle the edges
        p_idx = torch.randperm(positive_edge.size(1))
        p = positive_edge[:, p_idx]
        n_idx = torch.randperm(neg_edges.size(1))
        n = neg_edges[:, n_idx]
        
        num_batches = (positive_edge.size(1) + batch_size - 1) // batch_size
        
        for batch_idx in range(num_batches):
            # Batch the edges
            start = batch_idx * batch_size
            end = min((batch_idx + 1) * batch_size, positive_edge.size(1))
            pos_edge_index = p[:, start:end]
            neg_edge_index = n[:, start:end]
            # Calculate the maximum node index in the entire graph
            max_node_index = max(pos_edge_index.max(), neg_edge_index.max())
            num_nodes = max_node_index + 1
            
            # Create train masks with the shape of the entire graph
            pos_edge_mask = torch.zeros((num_nodes, num_nodes), dtype=torch.uint8)
            neg_edge_mask = torch.zeros((num_nodes, num_nodes), dtype=torch.uint8)

            # Set the positive and negative edge masks to 1 where edges exist
            pos_edge_mask[pos_edge_batch[0], pos_edge_index[1]] = 1
            neg_edge_mask[neg_edge_batch[0], neg_edge_index[1]] = 1

            # Make predictions and labels
            opt.zero_grad()
            pred = model(x, pos_edge_mask, neg_edge_mask)

            # Filter predictions and labels using the train mask
            train_pred = pred[train_edge_mask]
            train_labels = edge_label[train_edge_mask]

            # Calculate binary cross-entropy loss
            loss = F.binary_cross_entropy_with_logits(train_pred, train_labels.view(-1, 1))
            loss.backward()
            opt.step()

        acc = accuracy(edge_feature, edge_label)  
        print(emb)
        print(f"Epoch: {epoch+1}, Loss: {loss.item():.4f}, Accuracy: {acc}")


In [29]:

'''import time
import networkx as nx
import numpy as np
import torch
import torch.optim as optim
from tqdm import trange
import pandas as pd
import copy

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.nn as pyg_nn

import matplotlib.pyplot as plt

def train(x, positive_edges, neg_edges, args, batch_size):
    # Build the model
    model = GNNStack(16, args.hidden_dim, 8, args)
    
    scheduler, opt = build_optimizer(args, model.parameters())

    # Starting the training
    for epoch in trange(args.epochs, desc="Training", unit="Epochs"):
        total_loss = 0
        model.train()
        
        # Shuffle the edges
        p_idx = torch.randperm(positive_edges.size(1))
        p = positive_edges[:, p_idx]
        n_idx = torch.randperm(neg_edges.size(1))
        n = neg_edges[:, n_idx]
        
        # Batch the edges
        pos_edge_index = p[:, :batch_size]
        neg_edge_index = n[:, :batch_size]
        train_edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=1)
        
        # Make predictions and labels
        opt.zero_grad()
        pred = model(x, train_edge_index, pos_edge_index, neg_edge_index)
        
        # Create the train mask
        train_edge_mask = torch.cat([torch.ones(batch_size), torch.zeros(batch_size)], dim=0)
        train_edge_mask = train_edge_mask[train_edge_index]
        
        # Filter predictions and labels using the train mask
        train_pred = pred[train_edge_mask]
        train_labels = train_edge_mask

        # Calculate binary cross-entropy loss
        loss = F.binary_cross_entropy_with_logits(train_pred, train_labels.view(-1, 1))
        loss.backward()
        opt.step()

        acc = accuracy(train_pred, train_labels)
        print(f"Epoch: {epoch+1}, Loss: {loss.item():.4f}, Accuracy: {acc}")

trained_node_embeddings = train(x, positive_edges, neg_edges, args, 10000)'''


'import time\nimport networkx as nx\nimport numpy as np\nimport torch\nimport torch.optim as optim\nfrom tqdm import trange\nimport pandas as pd\nimport copy\n\nfrom torch_geometric.datasets import TUDataset\nfrom torch_geometric.datasets import Planetoid\nfrom torch_geometric.data import DataLoader\n\nimport torch_geometric.nn as pyg_nn\n\nimport matplotlib.pyplot as plt\n\ndef train(x, positive_edges, neg_edges, args, batch_size):\n    # Build the model\n    model = GNNStack(16, args.hidden_dim, 8, args)\n    \n    scheduler, opt = build_optimizer(args, model.parameters())\n\n    # Starting the training\n    for epoch in trange(args.epochs, desc="Training", unit="Epochs"):\n        total_loss = 0\n        model.train()\n        \n        # Shuffle the edges\n        p_idx = torch.randperm(positive_edges.size(1))\n        p = positive_edges[:, p_idx]\n        n_idx = torch.randperm(neg_edges.size(1))\n        n = neg_edges[:, n_idx]\n        \n        # Batch the edges\n        pos_ed

In [30]:
class objectview:
    def __init__(self, d):
        self.__dict__ = d

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    for args in [
        {'model_type': 'GraphSage', 'num_layers': 2, 'heads': 1, 'batch_size': 32, 'hidden_dim': 32, 'dropout': 0.5, 'epochs': 500, 'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0, 'weight_decay': 5e-3, 'lr': 0.001},
    ]:
        args = objectview(args)
        for model in ['GraphSage']:
            args.model_type = model

            # Match the dimension.
            if model == 'GAT':
                args.heads = 2
            else:
                args.heads = 1
                
trained_node_embeddings = train(x,positive_edges,neg_edges,args,1000)


Training:   0%|                                     | 0/500 [00:00<?, ?Epochs/s]